In [6]:
import pandas as pd
import torch

In [7]:
df = pd.read_csv('추자도.csv', encoding='cp949')
df

,지점,일시,평균 풍속(m/s),평균기압(hPa),평균 상대습도(%),평균 기온(°C),평균 수온(°C),평균 최대 파고(m),평균 유의 파고(m),최고 유의 파고(m),최고 최대 파고(m),평균 파주기(sec),최고 파주기(sec)
0,22184,2014-01-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.9,NaN,4.3
1,22184,2014-01-16,4.2,1029.6,60.0,4.4,14.6,1.0,0.6,0.9,1.4,3.3,4.3
2,22184,2014-01-17,3.7,1026.9,59.0,6.1,14.8,0.7,0.4,0.7,1.1,3.3,9.1
3,22184,2014-01-18,8.8,1028.5,66.0,5.8,14.9,2.8,1.8,2.7,4.2,5.3,7.1
4,22184,2014-01-19,3.8,1029.1,66.0,4.6,14.8,1.6,1.0,2.0,3.4,5.6,7.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3076,22184,2022-12-27,6.8,1028.8,63.0,5.6,16.1,1.3,0.8,1.0,1.5,3.7,4.6
3077,22184,2022-12-28,7.2,1030.4,59.0,5.6,16.0,1.6,1.0,1.3,2.3,4.5,5.8
3078,22184,2022-12-29,6.5,1032.6,59.0,4.9,16.1,1.5,0.9,1.1,1.9,4.7,6.4
3079,22184,2022-12-30,6.9,1032.3,53.0,5.8,16.2,1.5,0.9,1.2,2.0,4.3,4.9


In [8]:
df = df.rename(columns={'일시': 'date', '평균 기온(°C)': 'Atemp(°C)', '평균 수온(°C)': 'Stemp(°C)', '평균 최대 파고(m)':'wave(m)','평균 풍속(m/s)':'Wind(m/s)','평균기압(hPa)':'Apress(hPa)','평균 상대습도(%)':'Moisture(%)'})
df=df.set_index(pd.to_datetime(df.date)) # Date를 인덱스로
df=df[['Wind(m/s)','Apress(hPa)','Moisture(%)','Atemp(°C)','Stemp(°C)','wave(m)']] # Column 재정리
df.dropna(subset=['Wind(m/s)','Apress(hPa)','Moisture(%)','Atemp(°C)','Stemp(°C)','wave(m)'], inplace=True)
df

,Wind(m/s),Apress(hPa),Moisture(%),Atemp(°C),Stemp(°C),wave(m)
date,,,,,,
2014-01-16,4.2,1029.6,60.0,4.4,14.6,1.0
2014-01-17,3.7,1026.9,59.0,6.1,14.8,0.7
2014-01-18,8.8,1028.5,66.0,5.8,14.9,2.8
2014-01-19,3.8,1029.1,66.0,4.6,14.8,1.6
2014-01-20,9.0,1022.0,64.0,7.6,14.6,3.3
...,...,...,...,...,...,...
2022-12-27,6.8,1028.8,63.0,5.6,16.1,1.3
2022-12-28,7.2,1030.4,59.0,5.6,16.0,1.6
2022-12-29,6.5,1032.6,59.0,4.9,16.1,1.5


In [9]:
df = df.loc['2013-01-01':]
df

,Wind(m/s),Apress(hPa),Moisture(%),Atemp(°C),Stemp(°C),wave(m)
date,,,,,,
2014-01-16,4.2,1029.6,60.0,4.4,14.6,1.0
2014-01-17,3.7,1026.9,59.0,6.1,14.8,0.7
2014-01-18,8.8,1028.5,66.0,5.8,14.9,2.8
2014-01-19,3.8,1029.1,66.0,4.6,14.8,1.6
2014-01-20,9.0,1022.0,64.0,7.6,14.6,3.3
...,...,...,...,...,...,...
2022-12-27,6.8,1028.8,63.0,5.6,16.1,1.3
2022-12-28,7.2,1030.4,59.0,5.6,16.0,1.6
2022-12-29,6.5,1032.6,59.0,4.9,16.1,1.5


In [10]:
import pymysql
from sqlalchemy import create_engine, types

# Establish a connection to MySQL
host = 'localhost'
user = 'root'
password = 'k404'
database = 'forecast'

df = df.reset_index()

conn = pymysql.connect(host=host, user=user, password=password, database=database)

# Create a SQLAlchemy engine
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')

# Convert the DataFrame to a SQL table with specified column types
table_name = 'chujado'
dtype = {'date': types.Date(), 'Wind (m/s)': types.Float(), 'Apress (hPa)': types.Float(), 'Moisture (%)': types.Float(), 'Atemp (°C)': types.Float(), 'Stemp (°C)': types.Float(), 'wave (m)': types.Float()}
df.to_sql(table_name, engine, if_exists='replace', index=False, dtype=dtype)

# Close the connection
conn.close()
